<a href="https://colab.research.google.com/github/PedroDubas/Captcha_recognition/blob/master/Decifrando%20Captchas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

modelin = Sequential()

modelin.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (28,28,1)))
modelin.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
modelin.add(MaxPool2D(pool_size=(2,2)))
modelin.add(Dropout(0.25))

modelin.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
modelin.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
modelin.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
modelin.add(Dropout(0.25))


modelin.add(Flatten())
modelin.add(Dense(256, activation = "relu"))
modelin.add(Dropout(0.25))
modelin.add(Dense(19, activation = "softmax"))

modelin.load_weights('best.hdf5')

# Predict the values from the validation dataset
Y_pred = modelin.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = classes)

def leCaptcha_A(caminho,treino = treino):
    img = cv2.imread(caminho, 0)
    th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 17, 2)

    ret2, th2 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    blur = cv2.GaussianBlur(img, (5, 5), 0)
    ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


    kernel = np.ones((3,3), np.uint8)
    dilation = cv2.dilate(th, kernel, iterations=1)
    dilation2 = cv2.dilate(th2, kernel, iterations=1)
    dilation3 = cv2.dilate(th3, kernel, iterations=1)


    erosion = cv2.erode(dilation, kernel, iterations=1)
    erosion2 = cv2.erode(dilation2, kernel, iterations=1)
    erosion3 = cv2.erode(dilation3, kernel, iterations=1)


    kernel = np.ones((3,1), np.uint8)
    dilation = cv2.dilate(erosion, kernel, iterations=1)
    dilation2 = cv2.dilate(erosion2, kernel, iterations=1)
    dilation3 = cv2.dilate(erosion3, kernel, iterations=1)

    titles4 = ['Original', 'Adaptive', "Otsu", 'Gaussian + Otsu']
    images4 = [img, dilation, dilation2, dilation3]
    
    for i in range(4):
        plt.imshow(images4[i], 'gray')
        plt.show()

    #Get the individual letters.
    x, y, w, h = 30, 12, 21, 36
    idx=0
    for  i in range(5):
        
        
        crop_img = dilation3[y:y+h, x:x+w]
        
        image_resized = resize(crop_img, (28.0, 28.0),anti_aliasing=False).astype("int32")
        rows,cols = image_resized.shape # gives dimensions for RGB array
        img_size = rows*cols
        img_1D_vector = image_resized.reshape(img_size)
        
        lista = img_1D_vector.tolist()
        lista.insert(0,str(caminho[-9:][i]))
        
        new_row = pd.DataFrame(columns=treino.columns,data=[lista]) 
        treino = treino.append(new_row,ignore_index=True)   
        #cv2.imwrite('caracteres/'+str(caminho[-9:][i])+".png",crop_img)
        #cv2.rectangle(dilation3, (x, y), (x + w, y + h), (0, 255, 0), 1)
        x += w
        idx +=1
        
        plt.subplot(330 + (i+1))
        plt.imshow(image_resized, 'gray')
        
        
    plt.show()
    titles3 = ['Original', 'Adaptive', "Otsu", 'Gaussian + Otsu']
    images3 = [img, dilation, dilation2, dilation3] #img, opening, opening2, opening3]

    
    plt.imshow(img, 'gray')
    plt.show()
    
    plt.imshow(dilation3, 'gray')
    plt.title('Contouring')
    plt.show()
    
    
    captcha = (treino.iloc[:,1:].values).astype('float32')
    captcha = captcha.reshape(-1,28,28,1)
    #Predict
    captcha_pred = model.predict(captcha)
    # Convert predictions classes to one hot vectors 
    captcha_pred_classes = np.argmax(captcha_pred,axis = 1)
    # Convert validation observations to one hot vectors
    captcha_true = np.argmax(Y_val,axis = 1) 
    
    resp =''
    for letra_pred in captcha_pred_classes:
        resp = resp+ str(classes[letra_pred])
    print('CAPTCHA ======> :' +resp)
    
    return resp

leCaptcha_A('samples/cgcgb.png')

